In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import glob
from keras.models import model_from_json

In [2]:
allanimals = [plt.imread(file) for file in glob.glob("../datasets/animal/*.jpg")]
allpersons = [cv.imread(file) for file in glob.glob("../datasets/person/*.jpg")]
allcones = [cv.imread(file) for file in glob.glob("../datasets/roadCones/*.jpg")]
allstopsigns = [cv.imread(file) for file in glob.glob("../datasets/stop_sign/*.jpg")]
allredlights = [cv.imread(file) for file in glob.glob("../datasets/trafficLight_complete/red/*.jpg")]
allbluelights = [cv.imread(file) for file in glob.glob("../datasets/trafficLight_complete/blue/*.jpg")]
allgreenlights = [cv.imread(file) for file in glob.glob("../datasets/trafficLight_complete/green/*.jpg")]

In [3]:
X_test_original = allanimals[:]
X_test_original.extend(allpersons[:])
X_test_original.extend(allcones[:])
X_test_original.extend(allstopsigns[:])
X_test_original.extend(allredlights[:])
X_test_original.extend(allbluelights[:])
X_test_original.extend(allgreenlights[:])
X_test_original = np.array(X_test_original)
# X_test = X_test_original
X_test = np.zeros((X_test_original.shape[0], X_test_original.shape[1]//2, X_test_original.shape[2]//2, X_test_original.shape[3]))
for i in range(0,X_test_original.shape[0]):
    X_test[i] = cv.resize(X_test_original[i], (0,0), fx=0.5, fy=0.5) 

y_test = []
for i in range(0,1497):
    y_test.append(0)
for i in range(0,2087):
    y_test.append(1)
for i in range(0,1623):
    y_test.append(2)
for i in range(0,652):
    y_test.append(3)
for i in range(0,336):
    y_test.append(4)
for i in range(0,335):
    y_test.append(5)
for i in range(0,320):
    y_test.append(6)
    
y_test = np.array(y_test, dtype="uint8")
print(X_test.shape)
print(y_test.shape)
X_test = X_test / 255.0

(6850, 112, 112, 3)
(6850,)


In [4]:
# load json and create model
json_file = open('object_detection_model10000.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("object_detection_model10000.json").expect_partial()
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
loaded_model.evaluate(X_test, y_test)

Loaded model from disk
215/215 [==============================] - 8s 18ms/step - loss: 4.2002e-05 - accuracy: 1.0000 


[4.200176044832915e-05, 1.0]

In [5]:
import matplotlib.pylab as plt
import cv2
import numpy as np

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    #channel_count = img.shape[2]
    match_mask_color = 255
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def drow_the_lines(img, lines):
    img = np.copy(img)
    blank_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(blank_image, (x1,y1), (x2,y2), (0, 255, 0), thickness=10)

    img = cv2.addWeighted(img, 0.8, blank_image, 1, 0.0)
    return img

# = cv2.imread('road.jpg')
#image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
def process(image):
    print(image.shape)
    height = image.shape[0]
    width = image.shape[1]
    region_of_interest_vertices = [
        (0, height-100),
        (0, height/3),
        (700, height/3),
        (700, height-100)
    ]
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    canny_image = cv2.Canny(gray_image, 100, 80)
    cropped_image = region_of_interest(canny_image,
                    np.array([region_of_interest_vertices], np.int32),)
    lines = cv2.HoughLinesP(cropped_image,
                            rho=6,
                            theta=np.pi/180,
                            threshold=285,
                            lines=np.array([]),
                            minLineLength=40,
                            maxLineGap=100)
    image_with_lines = drow_the_lines(image, lines)
    return image_with_lines

cap = cv2.VideoCapture('../datasets/rpi-video-capture.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    frame = process(frame)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280